## Introduction

In this notebook we will propose you some example of how to run experiments with python-microscope.

These are just examples intended for training. In most cases you will want to use a full featured software package, 
with a dedicated GUI, like cockpit-microscope to run your experiments.

We are going to be running these example experiments using simulated devices. But the beauty of python-microscope 
is that the same code can be used to run experiments with real devices once they are connected to the PC. 

## Preparing the devices

Before we start, we need to prepare the devices. We will use a simulated camera, a simulated stage and a simulated
light source.

In [ ]:
from microscope.simulators import SimulatedCamera, SimulatedStage, SimulatedLightSource
from microscope import AxisLimits

from microscope.cameras.ximea import XimeaCamera
from microscope.controllers.asi import ASIMS2000

import numpy as np
from time import sleep
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

camera = XimeaCamera()

controller = ASIMS2000(port='/dev/ttyUSB0', baudrate=115200, lights=["465", "560"])

stage = controller.devices["stage"]
light_source = controller.devices["465"]

## A simple z-stack

Let's start with a simple z-stack. Let's start to define some parameters for the experiment.

In [ ]:
z_step_size = 0.35 # microns
# python-microscope stage do not use physical units. Instead, they use "counts". The number of counts per micron
# depends on the stage, the encoder it uses, the microstepping, etc. For a simulated stage, this does not really matter, 
# but for a real stage, you will need to know this number. You will find how to do so in the specific class implementation
z_counts_per_micron = 1
nr_of_z_steps = 20
exposure_time = 0.1 # seconds
light_power = 0.5 # as a fraction of the maximum power of the light source

camera.set_roi((512, 512, 1024, 1024))

Now we can start to run the experiment.

We are going to get the images from the camera using a queue. Qeues are interesting if you want to get the images from another thread

In [ ]:
# let's set the camera exposure time
camera.set_exposure_time(exposure_time)

# We first import the queue module and create a buffer
import queue

buffer = queue.Queue()

# and we tell the camera to put any new image into this buffer
camera.set_client(buffer)

# Let's enable the camera so it gets ready to acquire images
camera.enable()

# Let's keep track of the current position, so we can come back to it once the experiment is done
pre_experiment_position = stage.position

# Let's move the stage to the starting position. 
# We will consider the current position to be the center of our stack, so we move half the number of steps
# We want to move in the negative direction, so we use a negative number of steps
half_z_stack = -nr_of_z_steps * z_step_size * z_counts_per_micron / 2
stage.move_by({"Z": half_z_stack})

# Let's loop over the z positions
for z_step in range(nr_of_z_steps):
    # Turn on the light
    light_source.power = light_power
    
    # Trigger the camera
    camera.trigger()
    sleep(exposure_time)
    
    # Turn off the light
    light_source.power = 0
    
    # Move the stage now in the positive direction
    stage.move_by({"Z": z_step_size * z_counts_per_micron})
    
# Let's move the stage back to the original position
stage.move_to(pre_experiment_position)

# Let's disable the camera
camera.disable()

Now all the images are in the buffer as 2D numpy arrays. We can start to get them out and putting them into a 3D numpy array.

In [ ]:
# Let's create a 3D numpy array to store the images and get them from the queue
images = list(buffer.queue)

image_3d = np.array([i for i in images])

print(image_3d.shape)

and now some jupyter magic to display the 3D image

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim((0, image_3d.shape[1]))
ax.set_ylim((image_3d.shape[2],0))

im = ax.imshow(image_3d[0,:,:])
plt.close()

def init():
    im.set_data(image_3d[0,:,:])
    return (im,)
    
def animate(i):
    data_slice = image_3d[i,:,:]
    im.set_data(data_slice)
    return (im,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=200, blit=True)

HTML(anim.to_html5_video())

## A simple tiling array
We may try to do a tiling array. Let's define some parameters.


In [ ]:
x_counts_per_micron = 10  # As for the z axis, this number depends on the stage
y_counts_per_micron = 10
x_tile_size = 1000  # microns
y_tile_size = 1000
array_size = 3  # number of tiles in each direction
exposure_time = 0.1 # seconds
light_power = 0.5 # as a fraction of the maximum power of the light source

Lets define a function too loop in x and y to acquire the images in a spiral pattern.

In [ ]:
# let's set the camera exposure time
camera.set_exposure_time(exposure_time)

# Let's enable the camera so it gets ready to acquire images
camera.enable()

# A list to store the images
images = []

# Let's keep track of the current position, so we can come back to it once the experiment is done
pre_experiment_position = stage.position

def spiral(n):
    x = y = 0
    dx = 0
    dy = -1
    for i in range(n**2):
        if (-n/2 < x <= n/2) and (-n/2 < y <= n/2):
            yield (pre_experiment_position["X"] + (x * x_counts_per_micron * x_tile_size) , 
                   pre_experiment_position["Y"] + (y * y_counts_per_micron * y_tile_size))
        if x == y or (x < 0 and x == -y) or (x > 0 and x == 1-y):
            dx, dy = -dy, dx
        x, y = x+dx, y+dy
        
for pos_x, pos_y in spiral(array_size):
    stage.move_to({"X": pos_x, "Y": pos_y})
    light_source.power = light_power
    camera.trigger()
    sleep(exposure_time)
    light_source.power = 0

camera.disable()
stage.move_to(pre_experiment_position)


Getting again the images from the buffer

In [ ]:
images = list(buffer.queue)
image_3d = np.array([i for i in images])
print(image_3d.shape)

and show them in a plot

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim((0, image_3d.shape[1]))
ax.set_ylim((image_3d.shape[2],0))

im = ax.imshow(image_3d[0,:,:])
plt.close()

def init():
    im.set_data(image_3d[0,:,:])
    return (im,)
    
def animate(i):
    data_slice = image_3d[i,:,:]
    im.set_data(data_slice)
    return (im,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=200, blit=True)

HTML(anim.to_html5_video())